<a href="https://colab.research.google.com/github/afrahhassan/afrahhassan/blob/main/assignment2_susan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---
# <div align="center"><font>  </font></div>
# <div align="center"><font> COSC 2793 Computational Machine Learning </font></div>
## <div align="center"> <font> Assignment 2: Machine Learning Project </font></div>
## <div align="center"> <font> S3930776 Afrah Hassan A Alshaikh Ali & S3569266 Chang Su </font></div>
---

- 1. Load Data into DataFrame (shape_df, type_df)
- 2. check the dataset
- 3. encode the labels
- 4. rescale & data format
- 5. split data
- 6. flatten
- 7. Basline models (X_train flattened, y_train_one-encoded)
- 8. CNN model
- 9. Data Augmentation
- 10. Check the performance


In [2]:
#Import Libraries


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!cp -r "/content/drive/MyDrive/Assignment2" .

# **1. Data Loading and Prepocessing**

## **1.1 Load the dataset**

In [5]:
import os
import cv2
import pandas as pd

def load_images_into_dataframes(data_dir):
    shape_data = []
    type_data = []

    # Iterate through shape folders
    for shape_folder in os.listdir(data_dir):
        shape_folder_path = os.path.join(data_dir, shape_folder)

        # Skip non-directory files
        if not os.path.isdir(shape_folder_path):
            continue

        # Iterate through sign type folders
        for sign_type_folder in os.listdir(shape_folder_path):
            sign_type_folder_path = os.path.join(shape_folder_path, sign_type_folder)

            # Skip non-directory files
            if not os.path.isdir(sign_type_folder_path):
                continue

            # Iterate through image files
            for image_file in os.listdir(sign_type_folder_path):
                image_path = os.path.join(sign_type_folder_path, image_file)

                # Check if the file is an image
                if not image_path.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
                    print(f"Skipping non-image file: {image_path}")
                    continue

                # Load the image
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

                # Check if the image is null
                if image is None:
                    print(f"Skipping null image: {image_path}")
                    continue

                # Append image and labels to the respective data lists
                shape_data.append((image_path, image, shape_folder))
                type_data.append((image_path, image, sign_type_folder))

    # Create DataFrames from the collected data
    shape_df = pd.DataFrame(shape_data, columns=['image_path', 'image', 'shape_label'])
    type_df = pd.DataFrame(type_data, columns=['image_path', 'image', 'type_label'])

    return shape_df, type_df

# Example usage
# data_dir = "trafficsigns_dataset"
data_dir = "/content/drive/My Drive/Assignment2/trafficsigns_dataset"
shape_df, type_df = load_images_into_dataframes(data_dir)

print("Shape DataFrame:")
shape_df.head()

print("\nType DataFrame:")
type_df.head()


Skipping non-image file: /content/drive/My Drive/Assignment2/trafficsigns_dataset/triangle/warning/.DS_Store
Shape DataFrame:

Type DataFrame:


,image_path,image,type_label
0,/content/drive/My Drive/Assignment2/trafficsig...,"[[60, 64, 47, 39, 46, 33, 30, 33, 19, 24, 15, ...",crossing
1,/content/drive/My Drive/Assignment2/trafficsig...,"[[73, 63, 59, 55, 46, 51, 52, 52, 61, 56, 57, ...",crossing
2,/content/drive/My Drive/Assignment2/trafficsig...,"[[255, 255, 255, 255, 255, 254, 254, 254, 255,...",crossing
3,/content/drive/My Drive/Assignment2/trafficsig...,"[[37, 31, 30, 31, 30, 42, 37, 36, 35, 37, 35, ...",crossing
4,/content/drive/My Drive/Assignment2/trafficsig...,"[[117, 88, 77, 96, 87, 72, 79, 91, 80, 78, 76,...",crossing


This function load_data_into_dataframe loads image data from a directory into a DataFrame. It iterates through the directory structure to find image files and assigns them labels based on the folder structure. Then, it creates a DataFrame with columns for image paths, shape labels, and sign labels. Finally, it returns the DataFrame containing the loaded data.

## **1.2 EDA**

In [ ]:
# Check the size of the Shape DataFrame
print("DataFrame Size:")
print(shape_df.shape)

# Display basic information about the Shape DataFrame
print("\nDataFrame Information:")
print(shape_df.info())

# Display descriptive statistics of the Shape DataFrame
print("\nDataFrame Descriptive Statistics:")
print(shape_df.describe())

DataFrame Size:
(3699, 3)

DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3699 entries, 0 to 3698
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image_path   3699 non-null   object
 1   image        3699 non-null   object
 2   shape_label  3699 non-null   object
dtypes: object(3)
memory usage: 86.8+ KB
None

DataFrame Descriptive Statistics:


In [ ]:
# Check the size of the Type DataFrame
print("DataFrame Size:")
print(type_df.shape)

# Display basic information about the Type DataFrame
print("\nDataFrame Information:")
print(type_df.info())

# Display descriptive statistics of the Type DataFrame
print("\nDataFrame Descriptive Statistics:")
print(type_df.describe())

In [ ]:
type_df['image']

In [ ]:
# Number of data points
num_data_points_1 = len(type_df)
print("Number of Data Points:", num_data_points_1)

num_data_points_2 = len(shape_df)
print("Number of Data Points:", num_data_points_2)

# Count of unique shapes
num_unique_shapes = shape_df['shape_label'].nunique()
print("Number of Unique Image Shapes:", num_unique_shapes)

# Count of unique types
num_unique_types = type_df['type_label'].nunique()
print("Number of Unique Image Types:", num_unique_types)

In [ ]:
# Number of each shape
shape_counts = shape_df['shape_label'].value_counts()
print("Number of Each Shape Type:")
print(shape_counts)

# Number of each type
type_counts = type_df['type_label'].value_counts()
print("\nNumber of Each Type:")
print(type_counts)

In [ ]:
import matplotlib.pyplot as plt
def plot_samples(data_df, num_samples=5):
    fig, axes = plt.subplots(1, num_samples, figsize=(15, 3))

    for i in range(num_samples):
        sample = data_df.sample()  # Sampling from the DataFrame passed as input
        image = sample['image'].values[0]  # Assuming 'image' contains the image data
        label = sample['shape_label'].values[0] if 'shape_label' in data_df.columns else sample['type_label'].values[0]
        axes[i].imshow(image, cmap='gray')  # Assuming grayscale images
        axes[i].set_title(label)
        axes[i].axis('off')

    plt.show()


# Plot sample images from the shape DataFrame
print("Samples from Shape DataFrame:")
plot_samples(shape_df)

# Plot sample images from the type DataFrame
print("Samples from Type DataFrame:")
plot_samples(type_df)


Convert the shape labels into numerical labels: This step creates the target vector y_shape containing numerical labels for each shape category.
Perform one-hot encoding on the shape labels:

## **1.3 Preprocess Data - Encoding**


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create LabelEncoder instance
label_encoder = LabelEncoder()

# Fit and transform the shape labels
shape_labels_encoded = label_encoder.fit_transform(shape_df['shape_label'])

# Add the encoded labels as a new column in shape_df
shape_df['encoded_shape_label'] = shape_labels_encoded

# Display the updated shape_df
shape_df.head()

In [ ]:
# Remove duplicates based on 'image_path' column
shape_df_unique = shape_df.drop_duplicates(subset='image')

# Print the shape before and after removing duplicates
print("Shape of shape_df before removing duplicates:", shape_df.shape)
print("Shape of shape_df after removing duplicates:", shape_df_unique.shape)


In [ ]:
# Assuming 'type_df' is your DataFrame containing type data

# Remove duplicates based on 'image_path' column
type_df_unique = type_df.drop_duplicates(subset='image')

# Print the shape before and after removing duplicates
print("Shape of type_df before removing duplicates:", type_df.shape)
print("Shape of type_df after removing duplicates:", type_df_unique.shape)


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create LabelEncoder instance
label_encoder = LabelEncoder()

# Fit and transform the type labels
type_labels_encoded = label_encoder.fit_transform(type_df['type_label'])

# Add the encoded labels as a new column in type_df
type_df['encoded_type_label'] = type_labels_encoded

# Display the updated type_df
type_df.head()

In [ ]:
# Count the unique number of encoded shape labels
num_unique_encoded_labels = shape_df['encoded_shape_label'].nunique()

# Print the number of unique encoded shape labels
print("Number of Unique Encoded Shape Labels:", num_unique_encoded_labels)

In [ ]:
# Count the unique number of encoded type labels
num_unique_encoded_labels = type_df['encoded_type_label'].nunique()

# Print the number of unique encoded type labels
print("Number of Unique Encoded Type Labels:", num_unique_encoded_labels)

In [ ]:
# Function to check the shape of images
def check_image_shape(df):
    for image in df['image']:
        print("Image shape:", image.shape)

# Check the shape of images in shape_df
check_image_shape(shape_df)

# Check the shape of images in shape_df
check_image_shape(type_df)

## **1.4 Split the Data into train, test and val sets**

- Rescaling and Formatting the Images

- Split the Data: Stratified sampling is a sampling method where the population is divided into homogeneous subgroups called strata, and then samples are randomly selected from each stratum in proportion to the population size of the stratum. This ensures that the sample represents the population's diversity more accurately.

In [ ]:
# from sklearn.model_selection import train_test_split
# import numpy as np

# # Split data into train and test sets for shape data
# X_shape_train, X_shape_test, y_shape_train, y_shape_test = train_test_split(
#     shape_df['image_path'], shape_df['encoded_shape_label'], test_size=0.2, stratify=shape_df['shape_label'])

# # Split train set into train and validation sets for shape data
# X_shape_train, X_shape_val, y_shape_train, y_shape_val = train_test_split(
#     X_shape_train, y_shape_train, test_size=0.25, stratify=y_shape_train)

# # Split data into train and test sets for type data
# X_type_train, X_type_test, y_type_train, y_type_test = train_test_split(
#     type_df['image_path'], type_df['encoded_type_label'], test_size=0.2, stratify=type_df['type_label'])

# # Split train set into train and validation sets for type data
# X_type_train, X_type_val, y_type_train, y_type_val = train_test_split(
#     X_type_train, y_type_train, test_size=0.25, stratify=y_type_train)

# # Convert to numpy arrays
# X_shape_train = np.array(X_shape_train)
# X_shape_val = np.array(X_shape_val)
# X_shape_test = np.array(X_shape_test)
# y_shape_train = np.array(y_shape_train)
# y_shape_val = np.array(y_shape_val)
# y_shape_test = np.array(y_shape_test)

# X_type_train = np.array(X_type_train)
# X_type_val = np.array(X_type_val)
# X_type_test = np.array(X_type_test)
# y_type_train = np.array(y_type_train)
# y_type_val = np.array(y_type_val)
# y_type_test = np.array(y_type_test)

In [ ]:
# # Print shapes of the data sets after splitting for shape data
# print("\nShape of X_shape_train:", X_shape_train.shape)
# print("Shape of X_shape_val:", X_shape_val.shape)
# print("Shape of X_shape_test:", X_shape_test.shape)
# print("Shape of y_shape_train:", y_shape_train.shape)
# print("Shape of y_shape_val:", y_shape_val.shape)
# print("Shape of y_shape_test:", y_shape_test.shape)

In [ ]:
# # Print shapes of the data sets after splitting for type data
# print("\nShape of X_type_train:", X_type_train.shape)
# print("Shape of X_type_val:", X_type_val.shape)
# print("Shape of X_type_test:", X_type_test.shape)
# print("Shape of y_type_train:", y_type_train.shape)
# print("Shape of y_type_val:", y_type_val.shape)
# print("Shape of y_type_test:", y_type_test.shape)

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into train and test sets for shape data
X_shape_train, X_shape_test, y_shape_train, y_shape_test = train_test_split(
    shape_df['image_path'], shape_df['encoded_shape_label'], test_size=0.2, stratify=shape_df['encoded_shape_label'])

# Split train set into train and validation sets for shape data
X_shape_train, X_shape_val, y_shape_train, y_shape_val = train_test_split(
    X_shape_train, y_shape_train, test_size=0.25, stratify=y_shape_train)

# Split data into train and test sets for type data
X_type_train, X_type_test, y_type_train, y_type_test = train_test_split(
    type_df['image_path'], type_df['encoded_type_label'], test_size=0.2, stratify=type_df['encoded_type_label'])

# Split train set into train and validation sets for type data
X_type_train, X_type_val, y_type_train, y_type_val = train_test_split(
    X_type_train, y_type_train, test_size=0.25, stratify=y_type_train)

This code will split the data into training and testing sets for each sign label, ensuring a balanced distribution of data across the splits.

In [ ]:
print("Shape of Shape Data:")
print("X_shape_train shape:", X_shape_train.shape)
print("X_shape_val shape:", X_shape_val.shape)
print("X_shape_test shape:", X_shape_test.shape)
print("y_shape_train shape:", y_shape_train.shape)
print("y_shape_val shape:", y_shape_val.shape)
print("y_shape_test shape:", y_shape_test.shape)

For Shape Data:

- X_shape_train: Training set features for shape classification
- X_shape_val: Validation set features for shape classification
- X_shape_test: Test set features for shape classification
- y_shape_train: Training set labels for shape classification
- y_shape_val: Validation set labels for shape classification
- y_shape_test: Test set labels for shape classification -

In [ ]:
print("\nShape of Type Data:")
print("X_type_train shape:", X_type_train.shape)
print("X_type_val shape:", X_type_val.shape)
print("X_type_test shape:", X_type_test.shape)
print("y_type_train shape:", y_type_train.shape)
print("y_type_val shape:", y_type_val.shape)
print("y_type_test shape:", y_type_test.shape)

For Type Data:

- X_type_train: Training set features for type classification
- X_type_val: Validation set features for type classification
- X_type_test: Test set features for type classification
- y_type_train: Training set labels for type classification
- y_type_val: Validation set labels for type classification
- y_type_test: Test set labels for type classification

## **1.5 Flatten the images**

Flattening the images is a necessary step when using a Multi-Layer Perceptron (MLP) model because MLPs require input data to be in the form of one-dimensional arrays or vectors.

Each image in the dataset is represented as a two-dimensional array (e.g., 28x28 pixels for grayscale images). When we feed these images into an MLP, we need to flatten them into one-dimensional arrays so that each pixel value becomes a separate input feature.

In [ ]:
import numpy as np
# from keras.preprocessing.image import ImageDataGenerator
from PIL import Image

# Function to flatten images
def flatten_images(X):
    flattened_images = []
    for image_path in X:
        image = Image.open(image_path)
        image = np.array(image)
        flattened_image = image.flatten()
        flattened_images.append(flattened_image)
    return np.array(flattened_images)

# Flatten the images for shape data in training and testing sets
X_shape_train_flat = flatten_images(X_shape_train)
X_shape_test_flat = flatten_images(X_shape_test)
X_shape_val_flat = flatten_images(X_shape_val)

# Flatten the images for type data in training and testing sets
X_type_train_flat = flatten_images(X_type_train)
X_type_test_flat = flatten_images(X_type_test)
X_type_val_flat = flatten_images(X_type_val)

In [ ]:
# Print shapes of the flattened data sets
print("\nShape of X_shape_train_flat:", X_shape_train_flat.shape)
print("Shape of X_shape_test_flat:", X_shape_test_flat.shape)
print("Shape of X_shape_val_flat:", X_shape_val_flat.shape)

In [ ]:
print("\nShape of X_type_train_flat:", X_type_train_flat.shape)
print("Shape of X_type_test_flat:", X_type_test_flat.shape)
print("Shape of X_type_val_flat:", X_type_val_flat.shape)

# **2. Shape Classification**



## **2.1 Baseline Model (MLP Model with Two Layers Hidden)**

In [ ]:
import matplotlib.pyplot as plt

def plot_learning_curve(train_loss, val_loss, train_metric, val_metric, metric_name='Accuracy'):
    plt.figure(figsize=(10,5))

    plt.subplot(1,2,1)
    plt.plot(train_loss, 'r--', label='Train')
    plt.plot(val_loss, 'b--', label='Validation')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend(loc='upper left')

    plt.subplot(1,2,2)
    plt.plot(train_metric, 'r--', label='Train')
    plt.plot(val_metric, 'b--', label='Validation')
    plt.xlabel("Epochs")
    plt.ylabel(metric_name)
    plt.legend(loc='upper left')

    plt.show()

In [ ]:
X_shape_train_flat.shape

In [ ]:
y_shape_train.shape

This baseline model provides a starting point for our investigation into classifying traffic sign shapes using machine learning algorithms.


In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# Define the MLP model
def build_mlp(input_shape, num_classes):
    model = Sequential([
        Dense(128, activation='sigmoid', input_shape=(input_shape,)),
        Dense(64, activation='sigmoid'),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Set input shape
input_shape = X_shape_train_flat.shape[1]

# Number of classes
num_classes = len(np.unique(y_shape_train))

# Build the MLP model
model_mpl_shape = build_mlp(input_shape, num_classes)

# Compile the model
model_mpl_shape.compile(loss='sparse_categorical_crossentropy',
                   optimizer='SGD',
                   metrics=['accuracy'])

# Print model summary
model_mpl_shape.summary()


In [ ]:
# Train the model with validation data
history_mlp_shape = model_mpl_shape.fit(X_shape_train_flat, y_shape_train, epochs=10, batch_size=32, validation_data=(X_shape_val_flat, y_shape_val))

In [ ]:
# Check if history object contains the correct keys
print(history_mlp_shape.history.keys())

# Evaluate the model on the train, validation, and test data
train_loss_mlp_shape, train_accuracy_mlp_shape = model_mpl_shape.evaluate(X_shape_train_flat, y_shape_train, verbose=1)
val_loss_mlp_shape, val_accuracy_mlp_shape = model_mpl_shape.evaluate(X_shape_val_flat, y_shape_val, verbose=1)
test_loss_mlp_shape, test_accuracy_mlp_shape = model_mpl_shape.evaluate(X_shape_test_flat, y_shape_test, verbose=1)

# Print the results
print("Train Accuracy (MLP) for Shapes:", train_accuracy_mlp_shape)
print("Train Loss (MLP)for Shapes:", train_loss_mlp_shape)
print("Validation Accuracy (MLP) for Shapes:", val_accuracy_mlp_shape)
print("Validation Loss (MLP) for Shapes:", val_loss_mlp_shape)
print("Test Accuracy (MLP) for Shapes:", test_accuracy_mlp_shape)
print("Test Loss (MLP) for Shapes:", test_loss_mlp_shape)


In [ ]:
# Plot learning curves using the custom function
plot_learning_curve(history_cnn_shape.history['loss'], history_cnn_shape.history['val_loss'],
                    history_cnn_shape.history['accuracy'], history_cnn_shape.history['val_accuracy'],
                    metric_name='Accuracy')


The first plot shows the training and validation loss for the MLP model over the epochs. The training loss decreases steadily over time, while the validation loss initially decreases and then starts to increase after a certain number of epochs. This indicates that the model is overfitting to the training data.

The second plot shows the training and validation accuracy for the MLP model over the epochs. The training accuracy increases steadily over time, while the validation accuracy initially increases and then starts to plateau after a certain number of epochs. This also indicates that the model is overfitting to the training data.

Overall, the plots show that the MLP model is not performing well on the validation and test data. This is likely due to overfitting. To improve the performance of the model, we can try using regularization techniques or reducing the number of epochs.


In [ ]:
import numpy as np
from sklearn.metrics import classification_report

# Predict classes for test data
y_pred_1 = np.argmax(model_mpl_shape.predict(X_shape_test_flat), axis=1)

# Generate classification report
report = classification_report(y_shape_test, y_pred_1)

# Print the classification report
print("Classification Report:\n", report)

MLP model for shape classification is achieving an accuracy around 47.6% on the training, validation, and test sets. However, the precision, recall, and F1-score for each class are quite low. Here's a breakdown of the results:

- Train Accuracy (MLP) for Shapes: 47.59%
- Train Loss (MLP) for Shapes: 1.2547
- Validation Accuracy (MLP) for Shapes: 47.57%
- Validation Loss (MLP) for Shapes: 1.2599
- Test Accuracy (MLP) for Shapes: 47.57%
- Test Loss (MLP) for Shapes: 1.2651



**Classification Report:**

- The precision, recall, and F1-score for each class indicate poor performance.

- The overall accuracy is around 48%, but the model struggles to correctly classify instances across different classes.

Given these results, it's apparent that the MLP model is not performing well for shape classification. Further analysis and potentially model adjustments are needed to improve its performance.

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Compute confusion matrix for test data
conf_mat_test_mlp = confusion_matrix(y_shape_test, y_pred_1)

# Plot confusion matrix for test data
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat_test_mlp, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix - Test Data')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


The confusion matrix for the test data reveals that the MLP model is struggling to correctly classify instances across different classes.

## **2.2 Optimized Model: Convolutional Neural Network (CNN)**

Now, we've chosen the second model, CNN, because it's particularly effective for image classification tasks like the one at hand. Convolutional Neural Networks (CNNs) are adept at learning hierarchical representations of images, capturing features at different levels of abstraction. By using convolutional and pooling layers, CNNs can efficiently extract spatial patterns from input images, making them well-suited for tasks where the spatial relationships between pixels are crucial for classification.

### **2.1 Baseline CNN Model**

In [ ]:
import cv2

# Initialize lists to store problematic image paths
problematic_paths = []

# Concatenate shape training, validation, and testing data
X_train_shape_cnn = []
for path in X_shape_train:
    image = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        problematic_paths.append(path)
    else:
        X_train_shape_cnn.append(image.reshape(28, 28, 1))

X_train_shape_cnn = np.array(X_train_shape_cnn)

X_val_shape_cnn = []
for path in X_shape_val:
    image = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        problematic_paths.append(path)
    else:
        X_val_shape_cnn.append(image.reshape(28, 28, 1))

X_val_shape_cnn = np.array(X_val_shape_cnn)

X_test_shape_cnn = []
for path in X_shape_test:
    image = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        problematic_paths.append(path)
    else:
        X_test_shape_cnn.append(image.reshape(28, 28, 1))

X_test_shape_cnn = np.array(X_test_shape_cnn)

# Concatenate type training, validation, and testing data
X_train_type_cnn = []
for path in X_type_train:
    image = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        problematic_paths.append(path)
    else:
        X_train_type_cnn.append(image.reshape(28, 28, 1))

X_train_type_cnn = np.array(X_train_type_cnn)

X_val_type_cnn = []
for path in X_type_val:
    image = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        problematic_paths.append(path)
    else:
        X_val_type_cnn.append(image.reshape(28, 28, 1))

X_val_type_cnn = np.array(X_val_type_cnn)

X_test_type_cnn = []
for path in X_type_test:
    image = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        problematic_paths.append(path)
    else:
        X_test_type_cnn.append(image.reshape(28, 28, 1))

X_test_type_cnn = np.array(X_test_type_cnn)

# Print the shapes of the concatenated data
print("Shape of X_train_shape_cnn:", X_train_shape_cnn.shape)
print("Shape of X_val_shape_cnn:", X_val_shape_cnn.shape)
print("Shape of X_test_shape_cnn:", X_test_shape_cnn.shape)

print("Shape of X_train_type_cnn:", X_train_type_cnn.shape)
print("Shape of X_val_type_cnn:", X_val_type_cnn.shape)
print("Shape of X_test_type_cnn:", X_test_type_cnn.shape)

# Print problematic paths
print("Problematic image paths:", problematic_paths)


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the CNN model
def build_cnn(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='sigmoid', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='sigmoid'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='sigmoid'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='sigmoid'),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Set input shape
input_shape = (28, 28, 1)  # Assuming images are grayscale

# Number of classes
num_classes = len(np.unique(y_shape_train))

# Build the CNN model
cnn_model_shape = build_cnn(input_shape, num_classes)


# Print model summary
cnn_model_shape.summary()


In [ ]:
# Compile the model
# cnn_model_shape.compile(loss='sparse_categorical_crossentropy',
#                   optimizer='SGD',
#                   metrics=['accuracy'])

# Compile the model
cnn_model_shape.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
# history_cnn_shape = cnn_model_shape.fit(X_train_shape_cnn, y_shape_train, epochs=10, batch_size=32, verbose=1)
# Fit the CNN model on the training and validation data
history_cnn_shape = cnn_model_shape.fit(X_train_shape_cnn, y_shape_train, epochs=10, batch_size=32, validation_data=(X_val_shape_cnn, y_shape_val))


In [ ]:
# Evaluate the model on the training data
train_loss_cnn_shape, train_accuracy_cnn_shape = cnn_model_shape.evaluate(X_train_shape_cnn, y_shape_train)

# Print the training accuracy and loss
print("Training Accuracy (CNN) for Shapes:", train_accuracy_cnn_shape)
print("Training Loss (CNN) for Shapes:", train_loss_cnn_shape)

# Evaluate the model on the validation data
val_loss_cnn_shape, val_accuracy_cnn_shape = cnn_model_shape.evaluate(X_val_shape_cnn, y_shape_val)

# Print the validation accuracy and loss
print("Validation Accuracy (CNN) for Shapes:", val_accuracy_cnn_shape)
print("Validation Loss (CNN) for Shapes:", val_loss_cnn_shape)

# Evaluate the model on the test data
test_loss_cnn_shape, test_accuracy_cnn_shape = cnn_model_shape.evaluate(X_test_shape_cnn, y_shape_test)

# Print the test accuracy and loss
print("Test Accuracy (CNN) for Shapes:", test_accuracy_cnn_shape)
print("Test Loss (CNN) for Shapes:", test_loss_cnn_shape)


In [ ]:
# Plot learning curves using the custom function
plot_learning_curve(history_cnn_shape.history['loss'], history_cnn_shape.history['val_loss'],
                    history_cnn_shape.history['accuracy'], history_cnn_shape.history['val_accuracy'],
                    metric_name='Accuracy')


In [ ]:
print("Shape of X_train_shape_cnn:", X_train_shape_cnn.shape)
print("Shape of y_shape_train:", y_shape_train.shape)


In [ ]:
X_shape_test.shape

In [ ]:
y_shape_test.shape

### **2.2 Tuning Parameters use GridSearchCV**

In [99]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Define the function to create the CNN model
def create_cnn_model(optimizer='adam', activation='relu'):
    model = Sequential([
        Conv2D(32, (3, 3), activation=activation, input_shape=(28, 28, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation=activation),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation=activation),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation=activation),
        Dense(num_classes, activation='softmax')
    ])
    model_cnn_tune.compile(loss='sparse_categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model_cnn_tune

# Create a KerasClassifier based on the CNN model function
cnn_shape_tune = KerasClassifier(build_fn=create_cnn_model, epochs=10, batch_size=32, verbose=0)

# Define the hyperparameters to search
param_grid = {
    'optimizer': ['adam', 'sgd'],
    'activation': ['relu', 'sigmoid']
}

# Perform grid search
grid = GridSearchCV(estimator=cnn_shape_tune, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train_shape_cnn, y_shape_train)

# Print the best parameters found
print("Best parameters: ", grid_result.best_params_)

ModuleNotFoundError: No module named 'keras.wrappers'

In [100]:
!pip uninstall tensorflow
!pip install tensorflow

Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.15.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? Y
  Successfully uninstalled tensorflow-2.15.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 779.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

## Data Augmentation


In [37]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Initialize ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rotation_range=20,  # Random rotation up to 20 degrees
    width_shift_range=0.1,  # Randomly shift images horizontally (10% of total width)
    height_shift_range=0.1,  # Randomly shift images vertically (10% of total height)
    horizontal_flip=True,  # Randomly flip images horizontally
    vertical_flip=True,  # Randomly flip images vertically
    zoom_range=0.1,  # Randomly zoom images (by up to 10%)
    fill_mode='nearest',  # Fill in missing pixels with nearest neighbor
    rescale=1.0/255,  # Rescale pixel values to range [0, 1]
    data_format='channels_last'  # Image data format: channels_last
)

# Combined lists for original and augmented data
combined_X_shape_images = []
combined_y_shape_labels = []

# Apply augmentation to the entire training data for shapes
total_augmented_shape_images = 0
augmentation_limit = 2  # Set a limit for the number of augmented images per original image

for img_path, label in zip(X_shape_train, y_shape_train):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load the image as grayscale using OpenCV
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    img = np.expand_dims(img, axis=0)  # Add batch size dimension
    augmented_images_ind_shape = []
    num_augmented_images = 0
    for augmented_img in datagen.flow(img, batch_size=1):
        augmented_images_ind_shape.append(augmented_img[0])
        num_augmented_images += 1
        if num_augmented_images >= augmentation_limit:
            break
    # Append the original image and its augmented versions to the combined lists
    combined_X_shape_images.append(img[0])  # Remove the extra dimension
    combined_y_shape_labels.append(label)
    combined_X_shape_images.extend(augmented_images_ind_shape)
    combined_y_shape_labels.extend([label] * len(augmented_images_ind_shape))
    total_augmented_shape_images += len(augmented_images_ind_shape)

# Check the shape and size of the combined images
print("Combined images shape:", combined_X_shape_images[0].shape)  # Shape of the first image
print("Number of combined images:", len(combined_X_shape_images))

# Check the size of the combined labels
print("Number of combined labels:", len(combined_y_shape_labels))



Combined images shape: (28, 28, 1)
Number of combined images: 6657
Number of combined labels: 6657


shape distribution plot for data augmentation

In [38]:
import matplotlib.pyplot as plt
from collections import defaultdict

# Extract unique shape types
shape_types = defaultdict(int)
for shape in shapes:
    shape_type = shape[:-1]  # Exclude the last dimension (channels)
    shape_types[shape_type] += 1

# Sort the shape types by counts
sorted_shape_types = sorted(shape_types.items(), key=lambda x: x[1], reverse=True)

# Extract the sorted shape types and counts
sorted_shapes = [str(shape_type) for shape_type, count in sorted_shape_types]
sorted_counts = [count for shape_type, count in sorted_shape_types]

# Plot the distribution
plt.figure(figsize=(12, 6))
plt.bar(range(len(sorted_shapes)), sorted_counts, color='skyblue')
plt.title('Distribution of Combined Image Shape Types by Count')
plt.xlabel('Image Shape Type')
plt.ylabel('Count')
plt.xticks(range(len(sorted_shapes)), sorted_shapes, rotation=45, ha='right')
plt.tight_layout()
plt.show()



NameError: name 'shapes' is not defined

type distribution plot for data augmentation

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the CNN model with data augmentation
def build_cnn_augmented(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='sigmoid', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='sigmoid'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='sigmoid'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='sigmoid'),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Set input shape
input_shape = (28, 28, 1)  # Assuming images are grayscale

# Number of classes
num_classes = len(np.unique(y_shape_train))

# Build the CNN model with data augmentation
cnn_model_augmented = build_cnn_augmented(input_shape, num_classes)

# Compile the model
cnn_model_augmented.compile(loss='sparse_categorical_crossentropy',
                            optimizer='SGD',
                            metrics=['accuracy'])

# Print model summary
cnn_model_augmented.summary()


import numpy as np

# Convert the input data to NumPy arrays
combined_X_shape_images = np.array(combined_X_shape_images)
combined_y_shape_labels = np.array(combined_y_shape_labels)

# Train the CNN model with augmented data
history_augmented = cnn_model_augmented.fit(combined_X_shape_images, combined_y_shape_labels, epochs=10, batch_size=32, verbose=1)


In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = cnn_model.evaluate(combined_X_shape_images, combined_y_shape_labels, verbose=1)

# Print the test accuracy
print("Test Accuracy:", test_accuracy)


In [ ]:
#  edge extraction on both train and test images (include augmented images)

## Edge Extraction

Edge extraction can be a useful feature extraction technique for image classification tasks, as it helps capture important structural information present in the images.

In [ ]:
combined_X_shape_images[0].shape


In [ ]:
X_shape_test.shape

In [ ]:
import cv2
import numpy as np

# Function to perform edge extraction on a single image
def edge_extraction(image):
    # Check if the input image is a NumPy array
    if not isinstance(image, np.ndarray):
        raise ValueError("Input image must be a NumPy array")

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(image, (3, 3), 0)

    # Check if the blurred image is of the correct data type
    if blurred.dtype != np.uint8:
        blurred = blurred.astype(np.uint8)

    # Perform Canny edge detection
    edges = cv2.Canny(blurred, 30, 150)

    return edges

# Convert combined_X_shape_images to a NumPy array if it's not already
combined_X_shape_images = np.array(combined_X_shape_images)

# Load images from paths in X_shape_test and convert them to NumPy arrays
X_shape_test_images = []
for image_path in X_shape_test:
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
    if img is None:
        print(f"Failed to load image from path: {image_path}")
    else:
        X_shape_test_images.append(img)

# Apply edge extraction to the train images
X_train_edges = [edge_extraction(image) for image in combined_X_shape_images]

# Apply edge extraction to the test images
X_test_edges = [edge_extraction(image) for image in X_shape_test_images]

# Convert lists to NumPy arrays
X_train_edges = np.array(X_train_edges)
X_test_edges = np.array(X_test_edges)

# Print the shapes of the edge-extracted images
print("Shape of train images after edge extraction:", X_train_edges.shape)
print("Shape of test images after edge extraction:", X_test_edges.shape)


In [ ]:
# import numpy as np
# from keras.models import Sequential
# from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# # Define the CNN model architecture
# def build_cnn_model(input_shape, num_classes):
#     model = Sequential([
#         Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
#         MaxPooling2D((2, 2)),
#         Conv2D(64, (3, 3), activation='relu'),
#         MaxPooling2D((2, 2)),
#         Flatten(),
#         Dense(128, activation='relu'),
#         Dense(num_classes, activation='softmax')
#     ])
#     return model

# # Reshape the input data to include the channel dimension
# X_train_edges = np.expand_dims(X_train_edges, axis=-1)
# X_test_edges = np.expand_dims(X_test_edges, axis=-1)

# # Set input shape (shape of the input images)
# input_shape = X_train_edges[0].shape

# # Number of classes (assuming it's known)
# num_classes = 10  # Adjust this according to your dataset

# # Build the CNN model
# cnn_model = build_cnn_model(input_shape, num_classes)

# # Compile the model
# cnn_model.compile(optimizer='SGD',
#                   loss='sparse_categorical_crossentropy',
#                   metrics=['accuracy'])

# # Print model summary
# cnn_model.summary()

# # Train the CNN model
# history = cnn_model.fit(X_train_edges, combined_y_shape_labels, epochs=10, validation_data=(X_test_edges, y_shape_test))


In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the CNN model architecture
def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='sigmoid', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='sigmoid'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='sigmoid'),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Reshape the input data to include the channel dimension
X_train_edges = np.expand_dims(X_train_edges, axis=-1)
X_test_edges = np.expand_dims(X_test_edges, axis=-1)

# Set input shape (shape of the input images)
input_shape = X_train_edges[0].shape

# Number of classes (assuming it's known)
num_classes = 10  # Adjust this according to your dataset

# Build the CNN model
cnn_model = build_cnn_model(input_shape, num_classes)

# Compile the model
cnn_model.compile(optimizer='SGD',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Print model summary
cnn_model.summary()

# Train the CNN model
history = cnn_model.fit(X_train_edges, combined_y_shape_labels, epochs=10, validation_data=(X_test_edges, y_shape_test))


In [ ]:
print("Shape of X_train_edges:", X_train_edges.shape)
print("Shape of X_test_edges:", X_test_edges.shape)


In [ ]:
# Evaluate the model on the test data
loss, accuracy = cnn_model.evaluate(X_test_edges, y_shape_test)

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')


In [ ]:
X_train_edges.shape

## Hyperparameters Grid for Optimization

# early stoping for overfitting - baseline
regulization and drop out
tuning lamba = 1,,2 ..
increase regulization and drop out
error analysis

In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Define a function to build the CNN model
def build_cnn_model(num_filters=32, kernel_size=(3, 3), pool_size=(2, 2), num_neurons=128, activation='relu'):
    model = Sequential([
        Conv2D(num_filters, kernel_size, activation=activation, input_shape=(28, 28, 1)),
        MaxPooling2D(pool_size),
        Flatten(),
        Dense(num_neurons, activation=activation),
        Dense(10, activation='softmax')
    ])
    modelhttps://colab.research.google.com/drive/1p3tPtwnc4SNITXCYGuXVW4ujqFTtZLjO#scrollTo=early_stoping_for_overfitting_baseline.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Ensure the shapes of features and labels are consistent
print("Shape of X_train_edges:", X_train_edges.shape)
print("Shape of y_shape_train:", y_shape_train.shape)

# Define the hyperparameters grid
param_grid = {
    'num_filters': [32, 64],
    'kernel_size': [(3, 3), (5, 5)],
    'pool_size': [(2, 2), (3, 3)],
    'num_neurons': [128, 256],
    'activation': ['relu', 'sigmoid']
}

# Create a KerasClassifier with the build_cnn_model function
keras_classifier = KerasClassifier(build_cnn_model, epochs=5, batch_size=32, verbose=0)

# Perform grid search
grid = GridSearchCV(estimator=keras_classifier, param_grid=param_grid, cv=3, verbose=1)
grid_result = grid.fit(X_train_edges, y_shape_train)

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


In [ ]:
# !pip install --upgrade tensorflow


## Regularization Techniques


## Learning Rate Scheduling

In this report, a Convolutional Neural Network (CNN) is employed as the baseline model for image classification tasks owing to its aptitude in handling visual data. CNNs are adept at automatically extracting relevant features from images through layers like convolutional and pooling layers, enabling them to discern intricate patterns, textures, and shapes. Their hierarchical learning approach facilitates the capturing of complex relationships within images, making them ideal for tasks like image classification. Additionally, CNNs have exhibited exceptional performance across various computer vision applications and are particularly robust to variations in input data, further solidifying their suitability as a baseline model for image classification endeavors.

This function iterates over each row in the DataFrame, loads the corresponding image using OpenCV, resizes it to 28x28 pixels, normalizes the pixel values, and stores the preprocessed images and labels in lists. Finally, it converts the lists to numpy arrays and adds a channel dimension for compatibility with Conv2D layers in Keras.

The fit_transform() method fits the encoder to the training labels and transforms them into encoded numerical labels. Then, the transform() method is used to encode the validation and test labels using the same encoder fitted on the training labels. This ensures consistency in label encoding across different datasets.

- `train_images` are the preprocessed images for training.
- `train_labels_encoded` are the encoded numerical labels corresponding to the training images.
- `epochs=10` specifies the number of training epochs.
- `validation_data=(val_images, val_labels_encoded)` specifies the validation data to evaluate the model's performance during training. `val_images` are the preprocessed images for validation, and `val_labels_encoded` are the encoded numerical labels corresponding to the validation images.

This code defines a convolutional neural network (CNN) model using the Sequential API. It consists of three convolutional layers followed by max-pooling layers to downsample the feature maps. The final convolutional layer is followed by a flatten layer to transition from convolutional to dense layers. Two dense layers are added for classification, with the output layer having softmax activation to output class probabilities. The model is compiled with the Adam optimizer, sparse categorical crossentropy loss function, and accuracy as the evaluation metric.

Enhancing the baseline CNN model by Dropout Regularization.

Explination:

Enhancing the baseline CNN model by incorporating techniques such as learning rate scheduling and data augmentation.

Explination:

In [ ]:
# Reshape the images to 2D arrays
train_images_2d = train_images.reshape(train_images.shape[0], -1)
test_images_2d = test_images.reshape(test_images.shape[0], -1)

# Define the pipeline with preprocessing steps and the model
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Add preprocessing steps if necessary
    ('model', RandomForestClassifier())  # RandomForestClassifier with your desired model
])

# Define hyperparameters grid for optimization
param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [None, 10, 20],
    # Add more hyperparameters to optimize as needed
}

# Perform grid search cross-validation to find the best hyperparameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=1, n_jobs=-1)
grid_search.fit(train_images_2d, train_labels_encoded)

# Get the best model and evaluate its performance
best_model = grid_search.best_estimator_
test_accuracy = best_model.score(test_images_2d, test_labels_encoded)
print("Optimized Model Test Accuracy:", test_accuracy)
